In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [50]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [149]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas.plotting as pd_plt
import matplotlib.colors as plt_colors
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.experimental import enable_iterative_imputer
from IPython.display import display_html
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, validation_curve, train_test_split, KFold,GridSearchCV, RepeatedKFold, RepeatedStratifiedKFold, TunedThresholdClassifierCV, cross_validate
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder, TargetEncoder
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer, make_column_transformer

### Область работы 2 (выполнение лучшей модели)

In [152]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [178]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv'# содержит только имя файла, без имен папок

In [180]:
# Блок обучения модели

In [181]:
df = pd.read_csv(path_train)
df = df.drop_duplicates()
df['y']=np.where(df['y']>=15, df['y']/10,df['y'])
df['z']=np.where(df['z']>=15, df['z']/10,df['z'])
df = df[df['carat'] <= 4]
df = df[df['table'] <= 90]
df = df[df['x'] <= 10]
df = df[df['y'] <= 10]
df = df[df['z'] <= 7]
axis_features = ['x','y','z']
num_features = ['carat']#,'depth','table']
y_train = np.array(df.price)
X_train = df.drop(columns=['price'])
axis_transformer = Pipeline(steps=[
    ('missing_num', KNNImputer(missing_values = 0, n_neighbors=7)), #IterativeImputer(missing_values=0, max_iter=20)
    ('polynom', PolynomialFeatures(3,include_bias=False, interaction_only = True)),
    ('scaler', RobustScaler())])
num = Pipeline(steps=[
    ('missing_num', KNNImputer(n_neighbors=7)),
    ('bin', KBinsDiscretizer(n_bins = 4, strategy = 'uniform', encode = 'onehot-dense')),
    ('scaler', StandardScaler())]) #RobustScaler()
clarity_map = [{
    'col':'clarity',##### Обратить внимание
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]
clarity_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', RobustScaler())
    ])    
color_map = [
  {'col': 'color',
   'mapping': {'D': 6, 'E': 5, 'F': 4, 'G': 3, 'H': 2, 'I': 1, 'J': 0}}
]
color_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy = 'most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=color_map)),
    ('scaler', RobustScaler()) #MinMaxScaler()
    ])    
cut_map = [
  {'col': 'cut',
   'mapping': {'Ideal': 4, 'Premium': 3, 'Very Good': 2, 'Good': 1, 'Fair': 0}}
]
cut_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy='most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=cut_map)),
    ('scaler', MinMaxScaler())
    ])   
CT = ColumnTransformer([
        ("axis_transformer", axis_transformer, axis_features),
        ("num", num, num_features),# убрать быстую разряженную 
        ("ordinal_map", clarity_transformer, ['clarity']),
        ("color_map", color_transformer, ['color']),
        ("cut_map", cut_transformer, ['cut'])
        ]).set_output(transform='pandas')

display(CT)
ct = CT.fit_transform(X_train)
knn_regressor = Pipeline(steps=[
  ('preproc', CT),
  ('estimator', KNeighborsRegressor(n_neighbors=7, weights='distance', p=1, metric='euclidean'))
])
knn_regressor.fit(X_train, y_train)

ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('missing_num',
                                                  KNNImputer(missing_values=0,
                                                             n_neighbors=7)),
                                                 ('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False,
                                                                     interaction_only=True)),
                                                 ('scaler', RobustScaler())]),
                                 ['x', 'y', 'z']),
                                ('num',
                                 Pipeline(steps=[('missing_num',
                                                  KNNImputer(n_neighbors=7)),
                                                 ('bin',
                                                  KBinsDiscretizer(encode='o...
                                                  OrdinalEncoder(mapping=[{'col': 'color',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': D    6
E    5
F    4
G    3
H    2
I    1
J    0
dtype: int64}])),
                                                 ('scaler', RobustScaler())]),
                                 ['color']),
                                ('cut_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'cut',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': Ideal        4
Premium      3
Very Good    2
Good         1
Fair         0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['cut'])])

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('axis_transformer',
                                                  Pipeline(steps=[('missing_num',
                                                                   KNNImputer(missing_values=0,
                                                                              n_neighbors=7)),
                                                                  ('polynom',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False,
                                                                                      interaction_only=True)),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['x', 'y', 'z']),
                                                 ('num',
                                                  Pipeline(steps=[('missing_num',
                                                                   KNNImputer(n_neighbors=7)),
                                                                  ('bin'...
                                                  ['color']),
                                                 ('cut_map',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'cut',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': Ideal        4
Premium      3
Very Good    2
Good         1
Fair         0
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['cut'])])),
                ('estimator',
                 KNeighborsRegressor(metric='euclidean', n_neighbors=7, p=1,
                                     weights='distance'))])

In [183]:
y_knn_pred = knn_regressor.predict(X_train)

In [184]:
# Блок предсказания с использованием тестового набора

In [185]:
X_test = pd.read_csv(path_test)

In [187]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict = knn_regressor.predict(X_test)
y_predict

array([9365.89491235, 1607.        , 2856.62013393, ..., 2423.43649546,
       1587.41761532, 1978.82852215])